In [5]:
import pandas as pd
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()


i=0 ## batch counter

n=10000
sum_len = 20
sum_len_max=200

text_len = 100
text_len_max = 600


def count_ent(content):
    if count_ent.counter%1000 ==0:
        print('total processed:',count_ent.counter)
    count_ent.counter += 1
    temp_c = 0
    ents = nlp(content).ents
    for ent in ents:
        if ent.label_ in ['ORG', 'PRODUCT', 'FAC','WORK_OF_ART','MONEY']:
            temp_c = temp_c+1
    return temp_c
count_ent.counter=0

In [6]:
chunks = pd.read_json("downloads/corpus-webis-tldr-17.json", lines=True, chunksize = n,encoding='utf-8')
for c in chunks:
    if i%n == 0:
        if i ==0:
            #df = c.loc[(c.content_len>text_len)&(c.summary_len>sum_len),['content','summary']].reset_index(drop=True)
            #df = c.loc[(c.content_len>=text_len)&(c.content_len<=text_len_max)&
            #           (c.summary_len>=sum_len)&(c.summary_len<=sum_len_max),
            #           ['content','summary','content_len','summary_len']].reset_index(drop=True)
            df = c.loc[(c.content_len>=text_len)&(c.content_len<=text_len_max)&
                       (c.summary_len>=sum_len)&(c.summary_len<=sum_len_max),
                       ['author','content','summary','content_len','summary_len','id','subreddit','subreddit_id','title']].reset_index(drop=True)
        #else:
            ### save file
        #    filename = 'corpus_'+str(int(i/n))+'.csv'
        #    df.to_csv(r'output/corpus/'+filename)
            ### restart pd
        #    df = c.loc[(c.content_len>text_len)&(c.summary_len>sum_len),['content','summary']].reset_index(drop=True)
    else:
        ### append new parts
        #df = df.append(c.loc[(c.content_len>text_len)&(c.summary_len>sum_len),['content','summary']], ignore_index=True)
        #df = df.append(c.loc[(c.content_len>=text_len)&(c.content_len<=text_len_max)&
        #               (c.summary_len>=sum_len)&(c.summary_len<=sum_len_max),
        #                     ['content','summary','content_len','summary_len']], ignore_index=True)
        df = df.append(c.loc[(c.content_len>=text_len)&(c.content_len<=text_len_max)&
                             (c.summary_len>=sum_len)&(c.summary_len<=sum_len_max),
                             ['author','content','summary','content_len','summary_len','id','subreddit','subreddit_id','title']]
                       ,ignore_index=True)
    if i % 50 ==0:
        print('currently processing: ', i)
    i=i+1
### save the last one
print('Done processing with length.')



currently processing:  0
currently processing:  50
currently processing:  100
currently processing:  150
currently processing:  200
currently processing:  250
currently processing:  300
currently processing:  350
Done processing with length.


In [7]:
lines = len(df)
lines

1341180

In [8]:
df[0:3]

,author,content,summary,content_len,summary_len,id,subreddit,subreddit_id,title
0,iamacannibal,Theres an entire small town under the lake by ...,I'll try and get some similar shots from lake ...,181,25,c6aveyw,AbandonedPorn,t5_2sh6t,NaN
1,leep420,I take a beta blocker for my heart condition t...,Butchered daughter to save the universe and wa...,219,27,c6c7tdy,AskReddit,t5_2qh1i,NaN
2,thegoodweretaken,I was hanging out with friends when suddenly t...,I dreamed about being in a white room where I ...,153,33,c6c8fu1,AskReddit,t5_2qh1i,NaN


In [9]:
df.to_csv(r'output/corpus/tldr_20_200_100_600.csv',index=False)

In [ ]:
per_batch = int(lines/10)
for i in range(10):
    df2 = df[per_batch*i:per_batch*(i+1)]
    df2.to_csv(r'output/corpus/part'+str(i)+'.csv',index=False)

In [ ]:
###### recognizing entities ##########
import time


for i in range(10):
    if i==0:
        continue
    start_t = time.time()
    df = pd.read_csv('output/corpus/part'+str(i)+'.csv')
    print('Processing batch: '+str(i))
    df['ent_count'] = df['content'].apply(lambda x : count_ent(x))
    df.to_csv(r'output/corpus/part'+str(i)+'.csv',index=False)
    end_t = time.time()
    print('spent time:',str(end_t-start_t))


In [ ]:
df.to_csv(r'output/corpus/part0.csv',index=False)

In [ ]:
df.loc[df.ent_count>2,['content','summary']].to_csv(r'output/corpus/part0_ent.csv',index=False)

In [ ]:
df = df.loc[df.ent_count>0,['content','summary','content_len','summary_len']].reset_index(drop=True)
print('Done filtering unrelated contents')

####### saving file ##############
#filename = 'corpus_last.csv'
df.to_csv(r'output/corpus/len200_300_filtered.csv',index=False) ### first round of filtering and processing
print('Done saving')

In [ ]:
# read the split parts of processed data

df = pd.read_csv('output/corpus/part0.csv')
for i in range(1,10):
    df = df.append(pd.read_csv('output/corpus/part'+str(i)+'.csv'))

In [ ]:
df = df.loc[df.ent_count>=2]

In [ ]:
df[:10]

In [ ]:
df['content_len'].describe(include='all').apply(lambda x: format(x, 'f'))

In [ ]:
df['summary_len'].describe(include='all').apply(lambda x: format(x, 'f'))

In [ ]:
df.to_csv('output/corpus/filtered.csv')

In [ ]:
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

In [ ]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
def clean_text(text, for_summary = False):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    if for_summary: ## remove puctuation for content
        text = re.sub(r'[_"\-%()|+&=*%:#@\[\]/]', ' ', text)
    else:
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\n', ' ', text)
    # Remove stopwords for contents, not summaries
    if not for_summary:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    one = re.sub(r"([eoEO])\1\1+",r"\1\1",text) ## if more than 2 consecutive e or o, remove them
    text = re.sub(r"([^eoEO])\1\1+",r"\1",one) ## if more than 2 consecutive other characters, remove them
    text = re.sub(' +', ' ',text) ## remove redundant spaces
    return text

In [ ]:
df = pd.read_csv('output/corpus/filtered.csv')
df['content'] = df['content'].apply(lambda x: clean_text(x))
df['summary'] = df['summary'].apply(lambda x: clean_text(x, for_summary=True))

In [ ]:
df = df.loc[:,['content','summary','content_len','summary_len','ent_count']]
df[:10]

In [ ]:
df.summary[1]

In [ ]:
df

In [ ]:
df.to_csv('output/corpus/processed_1.csv',index=False)

In [ ]:
###################### read

import pandas as pd
import re

In [ ]:
df = pd.read_csv('output/corpus/processed_1.csv')

In [ ]:
df.summary[0]

In [ ]:
def space_before_punct(text):
    text = re.sub("([,.!?;$])",r" \1",text)
    text = re.sub(' +', ' ',text)
    return text

In [ ]:
df.summary = df.summary.apply(lambda x: space_before_punct(x))

In [ ]:
df[:10]

In [ ]:
df.to_csv('output/corpus/processed_2.csv',index=False)

In [ ]:
df[:1000].to_csv('output/corpus/toy.csv',index=False)

In [ ]:
import pandas as pd
import re

df = pd.read_csv('output/corpus/processed_2.csv')

def clean_text_again(text):
    text = re.sub(r'[^\x00-\x7F]+','', text) ### remove non-ascii
    return re.sub('[\^~<>]+','',text)

df.summary[0]

In [ ]:
df.content = df.content.apply(lambda x: clean_text_again(x))
df.summary = df.summary.apply(lambda x: clean_text_again(x))

In [ ]:
df.summary[1]

In [ ]:
df.content[1]